# Let's see if there is difference in convolutions' multiplications time on GPU depending on the format of the data (both input images and matrices weights).

In [1]:
import sys

import numpy as np
import os
import blosc

import tensorflow as tf
import matplotlib.pyplot as plt

import os
import blosc
import time

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
sys.path.append("..")


from dataset import DatasetIndex, Dataset

from dataset import Batch, action, model, inbatch_parallel, ImagesBatch

In [2]:
%matplotlib inline
# %env CUDA_VISIBLE_DEVICES = 1

In [57]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12010139642555102748, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 14680064
 locality {
   bus_id: 1
 }
 incarnation: 6580488357207528323
 physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0"]

In [144]:
def many_convs(d_type):
    if d_type == 'float64':
        d_type = tf.float64
    if d_type == 'float32':
        d_type = tf.float32
    
    x = tf.placeholder(d_type, [None, 256, 256])

    x_as_pics = tf.reshape(x, shape=[-1, 256, 256, 1])
    net = tf.layers.conv2d(x_as_pics, filters=3, kernel_size=(5,5), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    net = tf.layers.conv2d(net, filters=3, kernel_size=(3,3), strides=(1, 1), kernel_initializer=tf.truncated_normal_initializer(0.0, 1, dtype = d_type), padding='same')
    return x, net

In [2]:
def many_convs_second(d_type):
    if d_type == 'float16':
        d_type = tf.float16
    if d_type == 'float32':
        d_type = tf.float32
    
    
    x = tf.placeholder(d_type, [None, 256, 256])

    x_as_pics = tf.reshape(x, shape=[-1, 256, 256, 1])
    kernel_conv = (5, 5)
    filter_shape = tuple(kernel_conv) + (1, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(x_as_pics, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
    
    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')


    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')

    kernel_conv = (3, 3)
    filter_shape = tuple(kernel_conv) + (3, ) + (3, )
    filter_weights = tf.Variable(tf.truncated_normal(shape=filter_shape, stddev=1, dtype=d_type))
    net = tf.nn.convolution(net, filter=filter_weights, padding='SAME')
 
    return x, net

In [3]:
%env TF_FP16_MATMUL_USE_FP32_COMPUTE="False"

env: TF_FP16_MATMUL_USE_FP32_COMPUTE="False"


In [4]:
imgs_rand_16 = np.random.normal(size=(150, 256, 256)).astype('float16')

In [120]:
def measure_fp16():
    times = []
    for i in range(10):
        first_graph = tf.Graph()
        with first_graph.as_default():
            sess = tf.Session()

            start_time = time.clock()
            with tf.device('/gpu:0'):
                x, net = many_convs_second('float16')
                sess.run(tf.global_variables_initializer())
                result = sess.run(net, feed_dict={x: imgs_rand_16})

            end_time = time.clock()
            times.append(end_time - start_time)
            # print ('time: %s  seconds' % (end_time - start_time))
    print (np.mean(np.array(times)))
    return np.mean(np.array(times))

In [121]:
t_1 = []
for i in range(10):
    t_1.append(measure_fp16())

1.00821609735
0.993104598879
0.997061764503
1.00150022427
1.16349286519
1.00389593719
1.0059263946
1.30398896502
1.57751176979
1.00265944373


In [122]:
print (np.mean(np.array(t_1)))
print (np.std(np.array(t_1)))

1.10573580605
0.184673158062


In [14]:
imgs_rand_32 = np.random.normal(size=(150, 256, 256)).astype('float32')

In [128]:
def measure_fp32():
    times = []

    for i in range(10):
        second_graph = tf.Graph()
        with second_graph.as_default():
            sess = tf.Session()

            start_time = time.clock()
            with tf.device('/gpu:0'):
                x, net = many_convs_second('float32')
                sess.run(tf.global_variables_initializer())
                result = sess.run(net, feed_dict={x: imgs_rand_32})

            end_time = time.clock()
            times.append(end_time - start_time)
            # print ('time: %s  seconds' % (end_time - start_time))
    print (np.mean(np.array(times)))
    return (np.mean(np.array(times)))

In [131]:
t_2 = []
for i in range(10):
    t_2.append(measure_fp32())

1.24069207458
1.86945583639
1.37160364526
1.9665473946
1.99382096062
1.84998049005
1.47892487391
1.02915600305
1.12878629854
1.23611824855


In [132]:
print (np.mean(np.array(t_2)))
print (np.std(np.array(t_2)))

1.51650858256
0.350867764753


In [98]:
def many_convs_third(d_type):
    if d_type == 'int8':
        d_type = tf.int8
    if d_type == 'int16':
        d_type = tf.int16
    
    
    x = tf.placeholder(d_type, [None, 500])
    
    
    n_outs = 256
    n_ins = x.shape.as_list()[-1]
    W = tf.placeholder(d_type, [500, 256])
    b = tf.placeholder(d_type, [n_outs])
    net = tf.matmul(x, W)
    
#     net = tf.nn.xw_plus_b(x, W,  biases=b)
    
#     W = tf.Variable(tf.random_uniform(minval=0, maxval=100, shape=[n_ins, n_outs], dtype=d_type))
#     net = tf.nn.xw_plus_b(x, W, biases=tf.Variable(tf.zeros([n_outs], dtype=d_type)))
# #     net = tf.layers.dense(x, 128, kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=100, dtype=d_type))

    return x, W, b, net

In [96]:
imgs_int8 = np.random.randint(0, 20, size=(150, 500)).astype('int8')

In [65]:
imgs_int8.shape

(150, 500)

In [99]:
times = []
for i in range(10):
    first_graph = tf.Graph()
    with first_graph.as_default():
        sess = tf.Session()

        start_time = time.clock()
        with tf.device('/gpu:0'):
            x, W, b, net = many_convs_third('int8')
            sess.run(tf.global_variables_initializer())
            bias = np.zeros(256, dtype=int8)
            weights = np.random.randint(0, 20, size=(500, 256)).astype('int8')
            result = sess.run(net, feed_dict={x: imgs_int8, W:weights, b:bias})

        end_time = time.clock()
        times.append(end_time - start_time)
        print ('time: %s  seconds' % (end_time - start_time))
print (np.mean(np.array(times)))

TypeError: Value passed to parameter 'a' has DataType int8 not in list of allowed values: float16, float32, float64, int32, complex64, complex128

In [100]:
def many_convs_third(d_type):
    if d_type == 'int8':
        d_type = tf.int32
    if d_type == 'int16':
        d_type = tf.int16
    
    
    x = tf.placeholder(d_type, [None, 500])
    
    
    n_outs = 256
    n_ins = x.shape.as_list()[-1]
    W = tf.placeholder(d_type, [500, 256])
    b = tf.placeholder(d_type, [n_outs])
    net = tf.matmul(x, W)
    
#     net = tf.nn.xw_plus_b(x, W,  biases=b)
    
#     W = tf.Variable(tf.random_uniform(minval=0, maxval=100, shape=[n_ins, n_outs], dtype=d_type))
#     net = tf.nn.xw_plus_b(x, W, biases=tf.Variable(tf.zeros([n_outs], dtype=d_type)))
# #     net = tf.layers.dense(x, 128, kernel_initializer=tf.random_uniform_initializer(minval=0, maxval=100, dtype=d_type))

    return x, W, b, net

In [101]:
times = []
for i in range(10):
    first_graph = tf.Graph()
    with first_graph.as_default():
        sess = tf.Session()

        start_time = time.clock()
        with tf.device('/gpu:0'):
            x, W, b, net = many_convs_third('int8')
            sess.run(tf.global_variables_initializer())
            bias = np.zeros(256, dtype=int8)
            weights = np.random.randint(0, 20, size=(500, 256)).astype('int8')
            result = sess.run(net, feed_dict={x: imgs_int8, W:weights, b:bias})

        end_time = time.clock()
        times.append(end_time - start_time)
        print ('time: %s  seconds' % (end_time - start_time))
print (np.mean(np.array(times)))

InvalidArgumentError: Cannot assign a device for operation 'MatMul': Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
	 [[Node: MatMul = MatMul[T=DT_INT32, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](Placeholder, Placeholder_1)]]

Caused by op 'MatMul', defined at:
  File "C:\Users\Dari\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Dari\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Dari\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Dari\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Dari\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Dari\Anaconda3\lib\site-packages\zmq\

In [107]:
a = tf.placeholder(tf.int8, shape=[2,2])
b = tf.placeholder(tf.int8, shape=[2,2])
c = tf.matmul(a, b)

my_sess = tf.Session()
my_sess.run(tf.global_variables_initializer())

TypeError: Value passed to parameter 'a' has DataType int8 not in list of allowed values: float16, float32, float64, int32, complex64, complex128

In [119]:
a = tf.placeholder(tf.int32, shape=[2,2])
b = tf.placeholder(tf.int32, shape=[2,2])
c = tf.matmul(a, b)
with tf.device('/gpu:0'):

    my_sess = tf.Session()
    my_sess.run(tf.global_variables_initializer())
    ar = np.array([[1,0], [0,1]], dtype=np.int8)
    br = np.array([[2, 2], [3, 3]], dtype=np.int8)
    print (my_sess.run(c, feed_dict={a:ar, b:br}))

[[2 2]
 [3 3]]
